In [2]:
import numpy as np, pandas as pd, time, os, subprocess, scipy as sp, re, seaborn as sns
import app_config, importlib, app_lib
import diffmap as dm
from sklearn.metrics.pairwise import euclidean_distances

# plot_data_df = all_data.obs
# plot_data_df[app_config.params['display_coordinates']['x']] = all_data.obsm['X_umap'][:,0]
# plot_data_df[app_config.params['display_coordinates']['y']] = all_data.obsm['X_umap'][:,1]

In [3]:
_data_path

NameError: name '_data_path' is not defined

# STRING v11 data

In [127]:
importlib.reload(app_config)
string_protein_names_df = pd.read_csv(app_config.params['string_ppi_names_path'], sep="\t", index_col=False)
protein_names_ndces = string_protein_names_df['protein_external_id']
prot_names_to_ndces = dict(zip(protein_names_ndces.values, protein_names_ndces.index))

<module 'app_config' from '/Users/akshay/github/coessentiality-browser/app_config.py'>

In [60]:
itime = time.time()
string_network_df = pd.read_csv(app_config.params['string_ppi_network_path'], sep=" ", index_col=False)
print("Network loaded. Time: {}".format(time.time() - itime))

p1s = string_network_df['protein1'].values
p2s = string_network_df['protein2'].values
col1 = np.zeros_like(p1s)
col2 = np.zeros_like(p2s)

for d in range(len(p1s)):
    col1[d] = prot_names_to_ndces[p1s[d]]
    col2[d] = prot_names_to_ndces[p2s[d]]

string_network_df['protein1'] = col1
string_network_df['protein2'] = col2
print("Protein indices replaced. Time: {}".format(time.time() - itime))

string_data = string_network_df.values[:, 2].astype(int)
string_row = string_network_df.values[:, 0].astype(int)
string_col = string_network_df.values[:, 1].astype(int)
string_network_mat = sp.sparse.coo_matrix((string_data, (string_row, string_col)), shape=(len(protein_names_ndces), len(protein_names_ndces)))
sp.sparse.save_npz(app_config.params['string_ppi_matrix_path'], string_network_mat)

Network loaded. Time: 7.410933017730713
Protein indices replaced. Time: 14.084905862808228


In [13]:
string_network_mat = sp.sparse.load_npz(app_config.params['string_ppi_matrix_path']).tocsr()
new_string_mat = sp.sparse.csr_matrix((len(point_names), len(point_names)))
string_mat = sp.sparse.csr_matrix((string_network_mat.shape[0]+1, string_network_mat.shape[0]+1))
string_mat[:string_network_mat.shape[0], :string_network_mat.shape[0]] = string_network_mat

<19566x19566 sparse matrix of type '<class 'numpy.int64'>'
	with 11759454 stored elements in Compressed Sparse Row format>

In [52]:
strnames = string_protein_names_df['preferred_name']

In [79]:
coess_to_string = []
for i in range(new_string_mat.shape[0]):
    ndces_found = np.where(strnames == point_names[i])[0]
    if len(ndces_found) > 0:
        coess_to_string.append(ndces_found[0])
    else:
        coess_to_string.append(len(strnames))
    if i%1000 == 0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [91]:
sp.sparse.save_npz(app_config.params['string_matrix_ascoess_path'], string_mat[np.array(coess_to_string), :][:, np.array(coess_to_string)])

# Cheng et al. Nat Comm. 2019 data
## Entrez IDs downloaded from https://www.genenames.org/download/custom/

In [74]:
entrez_cheng_df = pd.read_excel('data/41467_2019_9186_MOESM3_ESM.xlsx')
entrez_IDs_df = pd.read_csv('data/hugo_entrez_ids.tsv', sep="\t", index_col=False)
entrez_to_genename = dict(entrez_IDs_df[['NCBI Gene ID', 'Approved symbol']].values)

In [76]:
plot_data_df = pd.read_csv(app_config.params['plot_data_df_path'][0], sep="\t", index_col=False)
point_names = np.array(plot_data_df['gene_names'])

In [99]:
itime = time.time()
j = 0
num_changes = 0
cheng_df = entrez_cheng_df.copy()
maskarr = np.zeros(cheng_df.shape[0])
for i in range(cheng_df.shape[0]):
    if (entrez_cheng_df.iloc[i, 0] in entrez_to_genename) and (entrez_cheng_df.iloc[i, 1] in entrez_to_genename):
        j += 1
        nadc = np.where(point_names == entrez_to_genename[entrez_cheng_df.iloc[i, 0]])[0]
        nadc2 = np.where(point_names == entrez_to_genename[entrez_cheng_df.iloc[i, 1]])[0]
        if (len(nadc) > 0) and (len(nadc2) > 0):
            cheng_df.iloc[i, 0] = nadc[0]
            cheng_df.iloc[i, 1] = nadc2[0]
            num_changes += 1
    if i%1000 == 0:
        print(i, j, time.time() - itime)

0 1 0.010242700576782227
1000 992 1.8789348602294922
2000 1983 3.7964179515838623
3000 2980 5.759443998336792
4000 3976 7.312947988510132
5000 4970 8.94202971458435
6000 5957 10.913097858428955
7000 6952 12.677075862884521
8000 7946 14.367710828781128
9000 8941 16.24048686027527
10000 9937 18.12824773788452
11000 10926 19.397721767425537
12000 11919 21.26417589187622
13000 12913 23.408305883407593
14000 13909 24.98733901977539
15000 14904 26.5570228099823
16000 15902 28.422478675842285
17000 16897 30.2503559589386
18000 17893 32.00580191612244
19000 18886 33.74929690361023
20000 19880 35.75053691864014
21000 20871 37.60000777244568
22000 21864 39.51102590560913
23000 22857 41.30736470222473
24000 23852 43.1773738861084
25000 24834 44.972580671310425
26000 25830 46.97437000274658
27000 26824 48.9245069026947
28000 27823 51.01590299606323
29000 28821 52.89437794685364
30000 29816 54.78169798851013
31000 30809 56.619364738464355
32000 31807 58.36479091644287
33000 32803 60.26641273498535


In [109]:
all_cheng_df = cheng_df.loc[np.sum(cheng_df == entrez_cheng_df, axis=1) == 1]

In [121]:
rows = []
cols = []
for i in range(all_cheng_df.shape[0]):
    rows.append(all_cheng_df.iloc[i, 0])
    cols.append(all_cheng_df.iloc[i, 1])
    rows.append(all_cheng_df.iloc[i, 1])
    cols.append(all_cheng_df.iloc[i, 0])
    if i%40000 == 0:
        print(i)

0
40000
80000
120000
160000


In [123]:
cheng_adj_mat = sp.sparse.coo_matrix((np.ones(len(rows)), (rows, cols)), shape=(point_names.shape[0], point_names.shape[0]))

In [128]:
sp.sparse.save_npz(app_config.params['cheng_matrix_path'], cheng_adj_mat)

In [10]:
x.tocoo().row

array([    0,     0,     0, ..., 17633, 17633, 17633], dtype=int32)

In [7]:
x=sp.sparse.load_npz(app_config.params['string_matrix_ascoess_path'])

# HuRI interactome data 
## (from http://interactome.baderlab.org/download)